In [ ]:
#This is the full pipeline for the text anaylsis
#There are two main components of analysis.
    #1. Persuasion detection 
    #2. Analysis to classify arguments

In [3]:
#Import statements
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas as pd
#import xgboost, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


# Collecting data from praw


In [1]:
import Praw
from nltk.tag import pos_tag
import text2emotion as te

PULL_SIZE = 3
subreddit = 'bitcoin'
data = Praw.scrapeToCsv(PULL_SIZE, subreddit)
data

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\acfuj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\acfuj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\acfuj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Version 7.4.0 of praw is outdated. Version 7.5.0 was released 7 days ago.
C:\Users\acfuj\Documents\GitHub\JuniorDesignML2\Pipeline\Praw.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dF[COMMENTS_CREATED] = dF[COMMENTS_CREATED].apply(lambda x: datetime.utcfromtimestamp(float(x)))


,Author,body,Created,Id,IsSubmitter,Link Id,Parent Id,Score,containsPersuasion
0,carl_von_linne,I would like to wish a lovely weekend to every...,2021-11-20 11:59:47,hldkrgd,False,t3_qxyf7w,t3_qxyf7w,13,-1
1,monaarts,All the BTC owning Thanksgiving travelers must...,2021-11-20 18:47:56,hleyp7k,False,t3_qxyf7w,t3_qxyf7w,14,-1
2,CapSignificant1078,This is the final dip before the next dip.,2021-11-20 16:04:55,hlebi6n,False,t3_qxyf7w,t3_qxyf7w,11,-1
3,loskubster,Now that’s a candle,2021-11-20 18:40:30,hlexn61,False,t3_qxyf7w,t3_qxyf7w,10,-1
4,MrKittenz,How soon till will see the next: MicroStrategy...,2021-11-20 05:48:54,hlctj2b,False,t3_qxyf7w,t3_qxyf7w,10,-1
...,...,...,...,...,...,...,...,...,...
956,Fotznbenutzernaml,"Yes mate, I am aware that property exists whic...",2021-11-21 03:19:53,hlgsmpj,False,t3_qycrmc,t1_hlgej8j,1,-1
957,php_questions,The same reason why you aren't borrowing money...,2021-11-21 02:31:07,hlgmzvl,False,t3_qycrmc,t1_hlgl6rc,4,-1
958,Nzwiebach,👀 we’re not borrowing money to buy bitcoin? I ...,2021-11-21 03:05:28,hlgqzih,False,t3_qycrmc,t1_hlgmzvl,1,-1
959,Evil__Maid,Oh yea good point,2021-11-21 02:32:54,hlgn7d9,False,t3_qycrmc,t1_hlgmzvl,2,-1


In [ ]:
#data = pd.read_csv("comments.csv")
#data = Praw.updateComments("comments.csv")
data

# Persuasion Detection

In [ ]:
#Part 1: Persuasion detection
#In this part, we are using a classified machine learning algorithm. It is
#Trained on ~70k reddit posts/comments that were gathered using PRAW

In [ ]:
#Setting up DataFrames

In [ ]:
data = pd.read_csv('etc/persuasionExamples6.csv', encoding = "latin1", engine='python', usecols=['body', 'containsPersuasion'])
data['containsPersuasion'] = np.where(data['containsPersuasion']=='[1]', 1, 0)
data = data.astype('U')
trainDF = pd.DataFrame()
trainDF['body'] = data['body']
trainDF['containsPersuasion'] = data['containsPersuasion']
data['containsPersuasion'].value_counts()

In [ ]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['body'], trainDF['containsPersuasion'])
train_x = train_x.astype('U')
valid_x = valid_x.astype('U')
train_y = train_y.astype('U')
valid_y = valid_y.astype('U')

encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [ ]:
#Setting up Count Vectorizer used in all 'XXCV' models.
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['body'])

xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [ ]:
#Setting up Word Vectorizer used in all 'XXWV' models.
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['body'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

In [ ]:
#Setting up N-gram Vectorizer used in all 'XXNV' models.
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['body'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

In [ ]:
#Training
def train_model(classifier, feature_vector_train, label):#, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    return classifier
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [ ]:
#Creates model using above function, notice which training sets are passed for which model
# Naive Bayes on Count Vectors
nbcv = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y)
predictions = nbcv.predict(xvalid_count)
print("NBCV: ", metrics.accuracy_score(predictions, valid_y))

# Naive Bayes on Word Level TF IDF Vectors
nbwv = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y)
predictions = nbwv.predict(xvalid_tfidf)
print("NBWV: ", metrics.accuracy_score(predictions, valid_y))

# Naive Bayes on Ngram Level TF IDF Vectors
nbnv = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y)
predictions = nbnv.predict(xvalid_tfidf_ngram)
print("NBNV: ", metrics.accuracy_score(predictions, valid_y))

In [ ]:
# Linear Classifier on Count Vectors
lrcv = train_model(linear_model.LogisticRegression(max_iter=1000000), xtrain_count, train_y)
predictions = lrcv.predict(xvalid_count)
print("LRCV: ", metrics.accuracy_score(predictions, valid_y))

# Linear Classifier on Word Level TF IDF Vectors
lrwv = train_model(linear_model.LogisticRegression(max_iter=1000000), xtrain_tfidf, train_y)
predictions = lrwv.predict(xvalid_tfidf)
print("LRWV: ", metrics.accuracy_score(predictions, valid_y))

# Linear Classifier on Ngram Level TF IDF Vectors
lrnv = train_model(linear_model.LogisticRegression(max_iter=1000000), xtrain_tfidf_ngram, train_y)
predictions = lrnv.predict(xvalid_tfidf_ngram)
print("LRNV: ", metrics.accuracy_score(predictions, valid_y))

In [5]:
#Manual testing, now that we have the classifiers trained, we can pass in our own tests.
#man_data = pd.read_csv('etc/testSet3.csv', encoding = "latin1", engine='python', usecols=['body', 'containsPersuasion'])
manData = [data["body"], data["containsPersuasion"]]
headers = ["body", "containsPersuasion"]
man_data = pd.concat(manData, axis = 1)
print(man_data)
man_x = man_data.body
man_y = man_data.containsPersuasion
#Have to use previous vectors.transform(man_x) to get right demensiosn.
man_x_cv = count_vect.transform(man_x)
man_x_wv = tfidf_vect.transform(man_x)
man_x_nv = tfidf_vect_ngram.transform(man_x)

[0      I would like to wish a lovely weekend to every...
1      All the BTC owning Thanksgiving travelers must...
2             This is the final dip before the next dip.
3                                    Now that’s a candle
4      How soon till will see the next: MicroStrategy...
                             ...                        
956    Yes mate, I am aware that property exists whic...
957    The same reason why you aren't borrowing money...
958    👀 we’re not borrowing money to buy bitcoin? I ...
959                                    Oh yea good point
960    Reddit saved me from bankruptcy, again. Thanks :)
Name: body, Length: 893, dtype: object, 0     -1
1     -1
2     -1
3     -1
4     -1
      ..
956   -1
957   -1
958   -1
959   -1
960   -1
Name: containsPersuasion, Length: 893, dtype: int64]


AttributeError: 'list' object has no attribute 'body'

In [ ]:
predictions = nbcv.predict(man_x_cv)
print("NBCV: ", metrics.accuracy_score(predictions, man_y))

In [ ]:
predictions = nbwv.predict(man_x_wv)
print("NBWV: ", metrics.accuracy_score(predictions, man_y))

In [ ]:
predictions = nbnv.predict(man_x_nv)
print("NBNV: ", metrics.accuracy_score(predictions, man_y))

In [ ]:
predictions = lrcv.predict(man_x_cv)
print("LRCV: ", metrics.accuracy_score(predictions, man_y))

In [ ]:
predictions = lrwv.predict(man_x_wv)
print("LRWV: ", metrics.accuracy_score(predictions, man_y))

In [ ]:
predictions = lrnv.predict(man_x_nv)
print("LRNV: ", metrics.accuracy_score(predictions, man_y))

In [ ]:
#Printing off confusion matrix of a specific algorithm
#[is not persuasive and guessed right, is not persuasive but guessed wrong]
#[Is persuasive but guessed wrong, is persuasicve and guessed right]
from sklearn.metrics import confusion_matrix
predictions = nbnv.predict(man_x_nv)
confusion_matrix = confusion_matrix(man_y, predictions)
print(confusion_matrix)

In [ ]:
#Part two: Analysis to classify argument 
#In this part, we take all examples that were marked as persuasivem and do
#Further analysis on them to estimate the classification of argument (Between 
#Logos, Ethos, and Pathos). Originally we wanted to do this via an unsupervised
#algorithm, but have switched to a range of manual tests.

In [ ]:
#Creats new dataframe containing only posts flagged as rhetoric
data = temp[temp['containsPersuasion'] == '[-1]'].copy()

In [ ]:
#Add If/Then/Num counts
data['If_Count'] = data['Body'].apply(lambda x: x.lower().count("if"))
data['Then_Count'] = data['Body'].apply(lambda x:x.lower().count('then'))
data['Num_Count'] = data['Body'].apply(lambda x: sum(c.isdigit() for c in x))

In [ ]:
#Add Counts for Proper Nouns
data['Num_Ethos_Keys'] = data['Body'].apply(lambda x:len([word for word, pos, in pos_tag(x.split()) if pos == 'NNP' or pos == 'NNPS']))

In [ ]:
#Perform Sentiment analysis on posts
anger, fear, happy, sad, surprise = [],[],[],[],[]
count = 0
for _, row in data.iterrows():
    if count % 100 ==0:
        print(count)
    count +=1
    emotions = te.get_emotion(row['Body'])
    anger.append(emotions.get("Angry"))
    fear.append(emotions.get("Fear"))
    happy.append(emotions.get("Happy"))
    sad.append(emotions.get("Sad"))
    surprise.append(emotions.get("Surprise"))
data["Anger_Score"]=anger
data["Fear_Score"]=fear
data["Happy_Score"]=happy
data["Sad_Score"]=sad
data["Surprise_Score"]=surprise

In [ ]:
data["Num_Logos_Keys"] = data['If_Count']+data['Then_Count']+data['Num_Count']
data.drop(columns=['If_Count', 'Then_Count', 'Num_Count'])
data

In [ ]:
data